In [1]:
import Base: reset

In [3]:
using Distributions, LightGraphs, ProgressMeter, RCall, DataFrames

In [4]:
srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.93472, 1.86877, 1.88112, 1.97201, 1.10737, 1.73105, 1.41204, 1.64686, 1.43909, 1.25583  …  1.36119, 1.19399, 1.48701, 1.16545, 1.73937, 1.94828, 1.00127, 1.94829, 1.7496, 1.95903], 382)

The grammar is a set of simple building blocks that generalize any diffusion simulation. 

 > `reset(N) -> seed(node_status, N, seed_fn, seed_size) -> 
            evolve(node_status, N, evolve_fn, ...) -> 
            sum(node_status)`  

The data structure on which diffusion simulations are executed is a combination of the network connections between the nodes and their *immutable* properties. All of them are initialized together at the beginning of each realization and is not tampered with in the course of the simulation. This data structure might hold different parameters for different kinds of models, for e.g., it might hold thresholds or influence of neighbors or influence of advertising. We illustrate our design using a simple simulation involving thresholds.  

In [5]:
struct Network
    G::LightGraphs.SimpleGraphs.SimpleGraph{Int}
    threshold::Vector{Float64}
end

### 1. Initialization

The starting point of any diffusion simulation is the `reset` function that takes a network object and sets the status of all the nodes in the network to `false`. The output is a BitVector.

In [6]:
function reset(N::Network)
    return falses(nv(N.G))
end

reset (generic function with 4 methods)

### 2. Seeding

The next step is to seed a subset of the network using a seeding function that dictates which specific nodes should be targeted. This mutates the state of the `node_status` vector in place. A purer version of this function would initialize a new vector that is the copy of `node_status`, update this vector and return a copy of the new vector. Let us define these two versions and check if there are any speed differences between the versions.

In [7]:
function seed!(node_status::BitVector, 
               N::Network,
               seed_fn::Function,
               seed_size::Int)
    
    seed = seed_fn(N, seed_size) # should return a vector of indices
    node_status[seed] = true
    
end

seed! (generic function with 1 method)

In [8]:
function seed(node_status::BitVector,
              N::Network,
              seed_fn::Function,
              seed_size::Int)
    
    new_status = copy(node_status)
    seed = seed_fn(N, seed_size) # should return a vector of indices
    new_status[seed] = true
    
    return new_status
end

seed (generic function with 1 method)

We illustrate the working of these functions and measure the timing using two example. 

#### 2.1 Random seeding

In [9]:
function seed_random(N::Network, seed_size::Int)
    return sample(vertices(N.G), seed_size, replace = false)
end

seed_random (generic function with 1 method)

In [10]:
N = Network(watts_strogatz(10^4, 3, 0.5), rand(TruncatedNormal(0.1, 0.05, 0, Inf), 10^4))

Network({10000, 10000} undirected simple Int64 graph, [0.134108, 0.0825621, 0.0878909, 0.0702239, 0.0238482, 0.00894452, 0.172579, 0.125979, 0.182814, 0.157071  …  0.0427552, 0.0175289, 0.0965888, 0.098674, 0.134587, 0.15765, 0.0636996, 0.140167, 0.0750944, 0.117351])

In [15]:
node_status = reset(N);

In [16]:
@time seed!(node_status, N, seed_random, 1000)

  0.000102 seconds (12 allocations: 86.656 KiB)


true

In [17]:
sum(node_status)

1000

In [18]:
node_status = reset(N);

In [21]:
@time node_status = seed(node_status, N, seed_random, 1000);

  0.000121 seconds (16 allocations: 88.047 KiB)


In [20]:
sum(node_status)

1000

#### 2.2 Seeding the nodes with highest pagerank centrality 

In [22]:
function seed_pagerank(N::Network, seed_size::Int)
    return sortperm(pagerank(N.G))[1:seed_size]
end

seed_pagerank (generic function with 1 method)

In [24]:
node_status = reset(N)
@time seed!(node_status, N, seed_pagerank, 1000)
sum(node_status)

  0.014962 seconds (22 allocations: 321.188 KiB)


1000

In [26]:
node_status = reset(N)
@time node_status = seed(node_status, N, seed_pagerank, 1000)
sum(node_status)

  0.013321 seconds (26 allocations: 322.578 KiB)


1000

This shows an interesting pattern when the bottleneck is shifted to the seeder algorithm. The more complex the seeding algorithm, lesser is the difference in speed between mutating and non-mutating updates.

By keeping the `seeder` function is independent of the `seed` function, we can quickly compare the relative benefits of the diffusion process across seeding methods.

### 3. Evolution

Different simulation design will employ different kinds of rules for node behavior. These rules are encoded into the `evolve_fn`. The `evolve_fn` function might be passed additional variables from its calller to support the evolution logic.

In [27]:
function evolve!(node_status::BitVector, 
                 N::Network,
                 evolve_fn::Function)
    
   for node in shuffle(vertices(N.G)) # by default nodes are updated in random order
        if node_status[node] == false
            evolve_fn(node_status, N, node)
        end
    end
end

evolve! (generic function with 1 method)

This higher level abstraction allows us to specify the evolution separate from the mechanics of evolving the diffusion process. The type and number of inputs to the evolver function can change depending on the complexity of the evolution rule.

We illustrate how the evolution works using a few example functions.

In [28]:
function evolve_threshold!(node_status::BitVector, N::Network, node::Int)
    
    n_engaged_nbrs = sum([node_status[nbr] for nbr in neighbors(N.G, node)])
    
    if n_engaged_nbrs/nv(N.G) > N.threshold[node] 
        node_status[node] = true
    end
end

evolve_threshold! (generic function with 1 method)

In [30]:
@time evolve!(node_status, N, evolve_threshold!)

  0.003739 seconds (27.01 k allocations: 1.312 MiB)


The `evolve` function is typically called multiple times in a typical simulation design.